In [30]:
import numpy as np
from numba import njit, prange
from tqdm import tqdm
import itertools

# Function to create a 4x4 transformation matrix from a rotation matrix and translation vector
@njit
def transformation_matrix(rotation, translation):
    T = np.eye(4)
    T[0:3, 0:3] = rotation
    T[0:3, 3] = translation
    return T

# Function to create a rotation matrix around x-axis
@njit
def rot_x(theta):
    arr1 = np.array([1, 0, 0])
    arr2 = np.array([0, np.cos(theta), -np.sin(theta)])
    arr3 = np.array([0, np.sin(theta), np.cos(theta)])
    return np.stack((arr1, arr2, arr3))
    
# Function to create a rotation matrix around z-axis
@njit
def rot_z(theta):
    arr1 = np.array([np.cos(theta), -np.sin(theta), 0])
    arr2 = np.array([np.sin(theta), np.cos(theta), 0])
    arr3 = np.array([0, 0, 1])
    return np.stack((arr1, arr2, arr3))

# Forward kinematics function
@njit
def fkin(angles):
    # Link lengths (assuming l1 = 0, l2 = l3 = l4 = l5 = l6 = 1)
    lengths = np.array([0, .2, .2, .2, .2, .2])

    # Define the translation vectors for each link
    links = [np.array([0, 0, lengths[i]]) for i in range(6)]

    # Rotation matrices for each joint
    R1 = rot_z(angles[0])
    R2 = rot_x(angles[1])
    R3 = rot_z(angles[2])
    R4 = rot_x(angles[3])
    R5 = rot_z(angles[4])
    R6 = rot_x(angles[5])

    # Transformation Matrices
    T1 = transformation_matrix(R1, links[0])
    T2 = transformation_matrix(R2, links[1])
    T3 = transformation_matrix(R3, links[2])
    T4 = transformation_matrix(R4, links[3])
    T5 = transformation_matrix(R5, links[4])
    T6 = transformation_matrix(R6, links[5])

    # Transformation for the end effector (assuming no additional translation)
    T7_end_eff = transformation_matrix(np.eye(3), np.array([0, 0, 1]))

    # Compute the overall transformation matrix
    T_final = T1 @ T2 @ T3 @ T4 @ T5 @ T6 @ T7_end_eff

    return T_final


# resolUtion of 18 bit
resolution = 2**15
# convert resolution to radians
res_rad = 2*np.pi/resolution

# calculate the error between the two methods
pos0 = fkin(np.zeros(6))[:3,3]
pos0_error = fkin(np.ones(6)*res_rad)[:3,3]
base_error = np.linalg.norm(pos0-pos0_error)

max_error = 0
final_config = None
for i in range(10000):
    config = np.random.rand(6)*2*np.pi
    config_pos = fkin(config)[:3,3]
    error_pos = fkin(config+np.ones(6)*res_rad)[:3,3]
    error = np.linalg.norm(config_pos-error_pos)
    if error > max_error:
        max_error = error
        final_config = config

print("base_error: ", base_error)
print("max_error: ", max_error)

base_error:  0.0008053398937404663
max_error:  0.0010039070093139381


In [29]:
from scipy import optimize
from scipy.optimize import basinhopping

def objective(q, sign=-1.0):
    q_error = q + np.ones(6)*res_rad
    pos = fkin(q)[:3,3]
    pos_error = fkin(q_error)[:3,3]
    error = np.linalg.norm(pos-pos_error)
    return sign*error

minimizer_kwargs = {"method": "BFGS"}
ret = basinhopping(objective, np.random.rand(6)*np.pi*2, minimizer_kwargs=minimizer_kwargs, niter=100)
ret.fun

-0.0010267312304340368